In [42]:
import warnings
warnings.filterwarnings('ignore')
from openai import OpenAI
from dotenv import load_dotenv
import os, openai
import yaml
from crewai import Agent, Task, Crew, Process

In [43]:
from pathlib import Path
env_path = '.env'
# Load the .env file
load_dotenv(dotenv_path=env_path)

True

In [44]:
OPEN_AI_KEY = os.getenv('OPEN_AI_KEY')
OPENAI_MODEL_NAME = 'gpt-4o-mini'

In [45]:
# Define file paths for YAML configurations
files = {
    'agents': 'config/agents.yaml',
    'tasks': 'config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

In [46]:
from pydantic import BaseModel, Field
from typing import List
from typing import List, Dict


class File(BaseModel):
    name: str = Field(..., description="Name of the file (e.g., 'models.py')")
    instructions: str = Field(..., description="Instructions for the file (e.g., 'Define database models here.')")

class App(BaseModel):
    name: str = Field(..., description="Name of the Django app (e.g., 'app1')")
    files: List[File] = Field(..., description="List of files within the app, with instructions")

class Project(BaseModel):
    name: str = Field(..., max_length=50, description="Name of the project (e.g., 'MyDjangoProject', max 50 characters)")
    description: str = Field(..., description="Short description of the project")
    requirements: List[str] = Field(..., description="List of required Python packages")
    env: Dict[str, str] = Field(..., description="Environment variables for the project")
    apps: List[App] = Field(..., description="List of apps in the project, each with its own files")

class Issue(BaseModel):
    category: str = Field(..., description="Category of the issue (e.g., MVP Alignment, Security, etc.)")
    description: str = Field(..., description="Detailed explanation of the issue")
    recommendation: str = Field(..., description="Recommended fix for the issue")

class QAReport(BaseModel):
    project_name: str = Field(..., description="Name of the project being reviewed")
    approval_status: str = Field(..., description="Approval status: 'Approved', 'Issues Detected', or 'Rejected'")
    issues: List[Issue] = Field(..., description="List of identified issues and recommendations")


In [47]:
architect= Agent(
  config=agents_config['architect'],
)
quality_manager= Agent(
  config=agents_config['quality_manager'],
)
script_agent= Agent(
  config=agents_config['script_agent'],
)


# Creating Tasks
design_project_task= Task(
    config=tasks_config['design_project_task'],
    agent=architect,
    output_pydantic=Project
)

review_design_task= Task(
    config=tasks_config['review_design_task'],
    agent=quality_manager,
    output_pydantic=QAReport
)

generate_script_task= Task(
    config=tasks_config['generate_script_task'],
    agent=script_agent,
    output_file="startup_create.sh"
)

backend_crew = Crew(
    agents=[
        architect,
        #quality_manager,
        script_agent
    ],
    tasks=[
        design_project_task,
        #review_design_task,
        generate_script_task
    ],
    process=Process.sequential,  # Sequential execution of tasks
    verbose = True,
    manager=True
)

Overriding of current TracerProvider is not allowed


In [48]:
mvp='''
A minimal Django MVP for your AI task-hiring platform would include these pieces:

User Management: Leverage Django authentication plus a custom Company profile model.
Task Model : Store task details (description, data, status, AI response).
AI Integration : A simple view or background process that calls an external AI API on new or updated tasks.
Basic UI : Templates or minimal front-end for:
Sign-up / login
Task creation
Status display and results
Admin Dashboard :Use Django  built-in admin for oversight of companies, tasks, and usage.
'''
crew_output = backend_crew.kickoff(inputs={'mvp':mvp})
print(crew_output)

# Agent: Django Project Architect
## Task: Using the provided MVP description, define the Django project structure. Include the following: - List of apps with their respective files and instructions. - Python package requirements. - Content for the .env file.
MVP: 
A minimal Django MVP for your AI task-hiring platform would include these pieces:

User Management: Leverage Django authentication plus a custom Company profile model.
Task Model : Store task details (description, data, status, AI response).
AI Integration : A simple view or background process that calls an external AI API on new or updated tasks.
Basic UI : Templates or minimal front-end for:
Sign-up / login
Task creation
Status display and results
Admin Dashboard :Use Django  built-in admin for oversight of companies, tasks, and usage.




# Agent: Django Project Architect
## Final Answer: 
{
  "name": "AI_Task_Hiring_Platform",
  "description": "A minimal Django application for managing task hiring leveraging AI integrati